In [65]:
import requests
import json
import pandas as pd
from pprint import pprint
import time
import numpy as np
from random import sample
from datetime import datetime
from config import yelp_key

In [3]:
data_file = "data/LOS_ANGELES_COUNTY_RESTAURANT_AND_MARKET_INSPECTIONS.csv"
data_file_pd = pd.read_csv(data_file)
data_file_cleaned = data_file_pd.drop(columns = ["OWNER ID", "RECORD ID", "PROGRAM NAME", "PROGRAM ELEMENT (PE)", "FACILITY STATE", "SERVICE CODE", "SERVICE DESCRIPTION", "SERIAL NUMBER", "EMPLOYEE ID"])
data_file_cleaned["ACTIVITY DATE"] = pd.to_datetime(data_file_cleaned["ACTIVITY DATE"])
data_file_cleaned.sort_values(by=['ACTIVITY DATE', 'FACILITY ID', 'FACILITY ADDRESS'], inplace=True, ascending=True)
data_file_cleaned.count()

ACTIVITY DATE       199950
OWNER NAME          199950
FACILITY ID         199950
FACILITY NAME       199950
PROGRAM STATUS      199950
PE DESCRIPTION      199950
FACILITY ADDRESS    199950
FACILITY CITY       199950
FACILITY ZIP        199950
SCORE               199950
GRADE               199895
dtype: int64

In [4]:
data_file_cleaned = data_file_cleaned.drop_duplicates(subset = ["FACILITY ID"], keep= 'last')
data_file_cleaned = data_file_cleaned.loc[(data_file_cleaned["PROGRAM STATUS"] == "ACTIVE")]
data_file_cleaned = data_file_cleaned[data_file_cleaned["PE DESCRIPTION"].str.contains("RESTAURANT")]
data_file_cleaned = data_file_cleaned.dropna(subset=['GRADE'])
data_file_cleaned["FULL ADDRESS"] = data_file_cleaned["FACILITY ADDRESS"] \
    + ", " + data_file_cleaned["FACILITY CITY"] + ", CA " + data_file_cleaned["FACILITY ZIP"] 
data_file_cleaned = data_file_cleaned[~data_file_cleaned['FACILITY NAME'].str.contains("#")]

data_file_cleaned.count()


ACTIVITY DATE       22512
OWNER NAME          22512
FACILITY ID         22512
FACILITY NAME       22512
PROGRAM STATUS      22512
PE DESCRIPTION      22512
FACILITY ADDRESS    22512
FACILITY CITY       22512
FACILITY ZIP        22512
SCORE               22512
GRADE               22512
FULL ADDRESS        22512
dtype: int64

In [5]:
sample_df = data_file_cleaned.sample(n=5000)
sample_df.count()



ACTIVITY DATE       5000
OWNER NAME          5000
FACILITY ID         5000
FACILITY NAME       5000
PROGRAM STATUS      5000
PE DESCRIPTION      5000
FACILITY ADDRESS    5000
FACILITY CITY       5000
FACILITY ZIP        5000
SCORE               5000
GRADE               5000
FULL ADDRESS        5000
dtype: int64

In [15]:
sample_df.to_csv("Sample_Restaurants_181216.csv", index=False)

In [25]:
sample_df = pd.read_csv("Sample_Restaurants_181216.csv", encoding="ISO-8859-1")
sample_df

,ACTIVITY DATE,OWNER NAME,FACILITY ID,FACILITY NAME,PROGRAM STATUS,PE DESCRIPTION,FACILITY ADDRESS,FACILITY CITY,FACILITY ZIP,SCORE,GRADE,FULL ADDRESS
0,2018-08-10,"CHENG, FLORA",FA0042891,PAGODA INN,ACTIVE,RESTAURANT (61-150) SEATS HIGH RISK,19348 RINALDI ST,PORTER RANCH,91326,70,C,"19348 RINALDI ST, PORTER RANCH, CA 91326"
1,2018-07-24,GLORIA BURGOS,FA0202740,TACOS EL SINALOENSE,ACTIVE,RESTAURANT (0-30) SEATS HIGH RISK,4813 E ROSECRANS AVE,COMPTON,90221-1851,87,B,"4813 E ROSECRANS AVE, COMPTON, CA 90221-1851"
2,2017-09-19,BOULEVARD LLC,FA0051336,THE DIME,ACTIVE,RESTAURANT (31-60) SEATS LOW RISK,442 N FAIRFAX AVE,LOS ANGELES,90036,93,A,"442 N FAIRFAX AVE, LOS ANGELES, CA 90036"
3,2018-07-17,ELITA KACHATRYAN/ARMINE SARGSY,FA0177733,AROMA BAKERY,ACTIVE,RESTAURANT (0-30) SEATS MODERATE RISK,12502 1/2 VANOWEN ST,NORTH HOLLYWOOD,91605,90,A,"12502 1/2 VANOWEN ST, NORTH HOLLYWOOD, CA 91605"
4,2018-06-13,GREAT AMERICAN CHICKENT CORP,FA0033130,KFC/TACO BELL,ACTIVE,RESTAURANT (31-60) SEATS MODERATE RISK,2941 W IMPERIAL HWY,INGLEWOOD,90303,95,A,"2941 W IMPERIAL HWY, INGLEWOOD, CA 90303"
5,2018-03-19,NORTAME INC,FA0251273,SUSHIMON,ACTIVE,RESTAURANT (31-60) SEATS HIGH RISK,3948 SEPULVEDA BLVD,TORRANCE,90505,91,A,"3948 SEPULVEDA BLVD, TORRANCE, CA 90505"
6,2018-08-14,EDWARD THOMAS HOSPITALITY CO,FA0008073,HOTEL CASA DEL MAR CATCH RESTAURANT,ACTIVE,RESTAURANT (61-150) SEATS HIGH RISK,1910 OCEAN FRONT WALK,SANTA MONICA,90401,96,A,"1910 OCEAN FRONT WALK, SANTA MONICA, CA 90401"
7,2018-07-19,"A & B AZUSA, INC",FA0249603,GRILL CITY - PREP KITCHEN,ACTIVE,RESTAURANT (0-30) SEATS HIGH RISK,2700 COLORADO BLVD,LOS ANGELES,90041,94,A,"2700 COLORADO BLVD, LOS ANGELES, CA 90041"
8,2018-07-09,CHOI JOON JAE,FA0035922,LE CAFE NOIR,ACTIVE,RESTAURANT (0-30) SEATS MODERATE RISK,500 S VIRGIL AVE STE 105,LOS ANGELES,90020,96,A,"500 S VIRGIL AVE STE 105, LOS ANGELES, CA 90020"
9,2018-08-30,SERGIO PAVON,FA0259294,THE FAMILY CAFE,ACTIVE,RESTAURANT (0-30) SEATS HIGH RISK,12121 WILSHIRE BLVD # 110,LOS ANGELES,90025,94,A,"12121 WILSHIRE BLVD # 110, LOS ANGELES, CA 90025"


In [8]:
headers = {"Authorization": "Bearer "+ yelp_key}
target_url = "https://api.yelp.com/v3/businesses/search"

final_df = pd.DataFrame()
name = []
rating = []
review_count = []
price = []

for index, row in sample_df.iterrows():
    params = {"term": row["FACILITY NAME"], "location": row["FULL ADDRESS"]}
    
    yelp_response = requests.get(target_url, params, headers=headers)
    
    if yelp_response.status_code != requests.codes.ok:
        print(yelp_response.text)
    else:
        yelp_data = yelp_response.json()
        
        try:
            name.append(yelp_data['businesses'][0]['name'])
            print(row['FACILITY NAME'])
        except (KeyError, IndexError):
            print("Missing Name...")
            name.append(row['FACILITY NAME'])
        try:
            rating.append(yelp_data['businesses'][0]['rating'])
            #print(yelp_data['businesses'][0]['rating'])
        except (KeyError, IndexError):
            #print("Missing Name...")
            rating.append("na")
        try:
            review_count.append(yelp_data['businesses'][0]['review_count'])
            #print(yelp_data['businesses'][0]['review_count'])
        except (KeyError, IndexError):
            #print("Missing Reviews...")
            review_count.append("na")
        try:    
            price.append(yelp_data['businesses'][0]['price'])
            #print(yelp_data['businesses'][0]['price'])
        except (KeyError, IndexError):
            #print("Missing Price...")
            price.append("na")
            
            


PAGODA INN
TACOS EL SINALOENSE
THE DIME
AROMA BAKERY
KFC/TACO BELL
SUSHIMON
HOTEL CASA DEL MAR CATCH RESTAURANT
GRILL CITY - PREP KITCHEN
LE CAFE NOIR
THE FAMILY CAFE
COMMONWEALTH
EMON JAPANESE RESTAURANT
EL ZORRO NIGHTCLUB
NUMERO UNO AGOURA HILLS
C & O TRATTORIA
THE BLVD
CAESAR PALACE
CAL MARE
LA FAVORITA BAKERY
HUDSON HOUSE
CAFE TRES LECHES
TACO BELL
BOILING POINT RESTAURANT HACIENDA HEIGHTS
LA INDIANA RESTAURANT
SUBWAY
CHA CHA CHICKEN
FREEZZONE RASPADOS
RAINBOW DONUTS
Missing Name...
RED IRON MONGOLIAN BBQ
THAI TIFFANY
SODEXHO MANAGEMENT INC
RINALDI'S OF REDONDO ITALIAN
Missing Name...
SAN YA BBQ RESTAURANT
INDIA'S GRILL
RINJANI
GREEN COTTAGE RESTAURANT
CASSIA RESTAURANT
Missing Name...
KEIKO GRILL
SCHOONERVILLE
R & B ROLL & KATSU
M & A AROMA
SUSHI RESTAURANT TAIHEI
SLOOPY'S BEACH CAFE
SUBWAY
THE RABBIT HOLE
Missing Name...
CHINA WOK
MISS DONUT & BAKERY
THE PARK'S FINEST
THE PIZZERIA FACTORY
GALAXY SPORT BAR
THANK U PHO
LUCKY LIZARD
D'AMORES NATURALLY
WILSHIRE COUNTRY CLUB
VICKY'S B

KC'S CAFE
SUSHI FUJI
BOB'S MARKET DELI/MEAT DEPT
CARL'S JR/GREEN BURRITO
TACO KINGS
1ST STREET BURGERS
HISAYA KYOTO CHESTNUTS CAFE
BASKIN ROBBINS ICE CREAM
Missing Name...
PATS COKTAILS
COCO FRESH TEA AND JUICE
RED ROCK
SWEET TIME DONUTS
HONEYFISH POKE
MIDORI SUSHI RESTAURANT
Missing Name...
LA NORIA RESTAURANT
TERMINAL 4 FOOD COURT KITCHEN
HOKKAIDO SEAFOOD BUFFET
MARISCOS SOL Y MAR
CO-OP NATURAL FOODS
Missing Name...
PANDA EXPRESS
COCOHODO
PF CHANG'S
HERA BAR CAFE
CHIPOTLE MEXICAN GRILL
HAPPY RICE
ABBY'S DINER
EL CANTINA DEL PATRON
TORTA SINALOA
DONUT INN
CREME DE LA CREPE
LUAU LARRY'S
ACAPULCO RESTAURANT
VINCE'S PASTA & PIZZA
PICO KOSHER DELI
MENCHIE'S FROZEN YOGURT
LAKEWOOD COUNTRY CLUB
MONGOLIAN HOT GRILL
TIA JUANA'S
BANQUET ROOM
90 WEST LOUNGE
BORAN THAI RESTAURANT
TORI MATSU RESTAURANT
THE PIE HOLE AMERICANA
O JANG DONG RESTAURANT
POKE BAR
KNOCKOUTS GENTLEMEN'S CLUB
DOMINO'S PIZZA
IT'S A GRIND
EARTHBAR MARINA DEL REY
IT'S THAI HOMETAURANT
DONUT KING
PICCOLO PARADISO
TENDER GREENS

PANZANELLA RISTORANTE
VIET HUONG
JJ BAKERY
MTN
IT'S BOBA TIME
KENTUCKY FRIED CHICKEN
PILI'S TACOS
COMMUNITY CENTER COFFEE SHOP HOLY GROUNDS
APOLLO DRIVE IN
THE WHITE HARTE
Missing Name...
Missing Name...
POLLO CAMPERO
NAPOLI PIZZERIA
SFV JAPANESE COMMUNITY CENTE
THE CAJUN BAYOU
TROPICAL FRUITS & JUICES
PORTA VIA
BUDNAMU KOREAN BBQ HOUSE
CARAMBA SPORTS BAR
GREEN HORSE
MEN OH
ISABELS BAKERY & CAFE, INC.
THE CHEESECAKE FACTORY
SHABUYA
BIRIYANI KABOB HOUSE
DOMINO'S PIZZA
TOP SUSHI
NEW YORK BAGEL CO
PINKBERRY MONROVIA
GARCIA'S FRUIT
PROTINI BAR
MA'KAI LOUNGE
DONUT GALORE
DOMINO'S PIZZA
ANTOJITOS CUQUIO
THE BLUE DOG
Missing Name...
DONUT KING
PRIME CUT BEEF JERKY
SUPER TACOS CULIACAN
TACO BELL
PANADERIA LA ANTIGUA
TOCAYA ORGANICA
DON CUCO'S MEXICAN REST.
THE LION CAFE
HAMASAKU
MARISCOS EL GRINGO
FAUBOURG RESTAURANT
FOOD AND BOUNTY
POKI YO
Missing Name...
JENNIFER PENNIFER
CAFE 50'S
Missing Name...
TAK'S COFFEE SHOP
CUVEE II
PIANO BAR
TRADINOI RESTAURANT
TACOS EL GRULLO
JK DONUTS
PEI WEI FRES

TUTTI FRUTTI FROZEN YOGURT
ELVIRA'S FINEST FOODS OF MEXICO
TSURUMARU UDON
HAMBONE'S BAR-B-CUE
THE ARTIST PIZZERIA
MCCONNELL'S FINE ICE CREAM
CAFFE ETC
SALATHAI THAI CUISINE
YANDICRIS BIONICOS & FRUIT
XI AN TASTY
NORAH RESTAURANT
LA COCINA DE PAQUITA
R-I CLUB LOUNGE
MEDIDATE COFFEE
MONTEREY PALACE RESTAURANT
MAKE OUT
PHO & ROLL
THAI CORNER
BON BON TEA HOUSE
CHILLIN THAI CUISINE
STARBUCKS
TOWER OF PIZZA INC
TACO NAZO
ENYA
Missing Name...
DRAGON PALACE
HAAGEN DAZS
JENI'S SPLENDID ICE CREAM
ROYAL DRAGON
FARMER BOYS
CARINOS
LA PUENTE BAKERY
SEAHAWK COCKTAIL LOUNGE
LA NUEVA FLOR BLANCA
Missing Name...
FULL HOUSE SEAFOOD RESTAURANT
PAPA JOHN'S PIZZA
OLYMPIC BURGER
EL TACO LOCO MEXICAN RESTAURANT
CHAN DARA
KOGIYA
MR YOU EXPRESS
WARNER GRAND THEATER
CALIFORNIA ROLL & SUSHI
MARVIN
GUADALAJARA GRILL
RAINBOW DONUTS AND ICE CREAM
AND WAFFLES
WONG JAVA HOUSE
FUENTE DE SALUD
TACOS MEXICO MARIA CONTRERAS
B.L. RESTAURANT
Missing Name...
KIKKA @ WHOLE FOODS TARZANA
THE BOILING CRAB
YUZU SHABU & GRILL
Mi

KNOX DELI
ORO'S PIZZA & PASTA
EL GALLO PINTO RESTAURANT
Missing Name...
EL NUEVO MARISCOS TAMPICO
MAYA'S GRILL
BURGERIM
CHIN GARDEN INC
DOMINO'S PIZZA
PLANET RESTAURANT
BUCHUN GAMJATANG RESTAURANT
DABANG DESIGN CAFE
CAKE AND ART BAKERY
LA PAZ BAKERY
REDTOP KITCHEN
TACO BELL
Missing Name...
MILLET / CHOCOLATE OPULENCE
THE HEART LODGE AND HONKY TONK
ZINC CAFE & MARKET
EL RIO BRAVO RESTAURANT
WA SUSHI & BISTRO
ORLEANS AND YORK DELI
ABC CHINESE FOOD
BIBIGO
IZAKAYA AKATORA
LA CABANA RESTAURANT
BIG SOFTEE
AYCE FOOD, INC.
MARRIOTT CORP
DISNEY CAFE BY LOVE CATERING
THAI DISH
BIRRIERIA APATZINGAN
Missing Name...
CHIPOTLE MEXICAN GRILL
MOONLIGHT TAVERN
KINGS KABOB
Missing Name...
TOASTED AND ROASTED
BIRRIERIA LALO'S
HOTEL CASA DEL MAR-EMPLOYEE CAFETERIA
BLUESTONE LANE
THE VEGGIE GRILL
GREEN ZONE
PATRICIO'S GRILL & PASTA
MISO SUSHI
CAFFE LUXXE
YAHA RESTAURANT
WOO REE RESTAURANT
EL 7 MARES/PESCADERIA
NO GHOST BEARS
Missing Name...
KAGURA EL SEGUNDO
ADMIRALS CLUB
SANTA FE CAFE
AMDA MARKET
CAPTAIN K

...

SOUTHLAND BEER
PIER EXPRESS
CAFE D'ETOILE
KOURAKU RESTAURANT
LITTLE BEACH HOUSE MALIBU
LOS BURRITOS
MOTHER MOO CREAMERY
Missing Name...
SPICES THAI KITCHEN
CAFETERIA PACO'S
AKU POKE
Missing Name...
A WON JAPANESE RESTAURANT
WAHOO CHAYA
DOMINO'S PIZZA
BLUE FISH RESTAURANT
LAWRY'S THE PRIME RIB
THE YORK
EL CERRO VERDE
SPORTSMEN'S LODGE
BIG MAMA'S & PAPA'S PIZZERIA
SWEET GREEN SANTA MONICA
VERNETTI
THE HABIT BURGER GRILL
PEPPERDINE UNIVERSITY/ MAIN CAFETERIA
CHINA WOK EXPRESS
SA RESTAURANT
EL POLLO SONORA
SABORES DEL PERU
CIELITO LINDO
CHINA WOK
TACO BELL / PIZZA HUT EXPRESS
LA PAPA LOCA
VILLA MORELIANA
GRAND STAR
QUICKLY
SODEXO CAFE
BAMBOO
DIANA'S RESTAURANT
DONUT DEN
Missing Name...
TACOS EL REY
LAS BRISAS
MONSTER BURGER
SBARRO
BOBA TIME
BODEGA WINE BAR
MOON WOK
N 2 GELATO STORE
EDIBLE ARRANGEMENTS
T- BAAR
CORNER BAKERY CAFE
POST &  BEAM
STORY TAVERN
18TH STREET CAFE
TEA LAB
LETTE
LOFT BAR AT POOL LEVEL
POPEYES CHICKEN
FARMER BOYS
GREEN GARDEN
HAPPY BOWL
THE COFFEE BEAN & TEA LEAF
DAW Y

PANDA EXPRESS
FLAMING PATTY'S
CALIFORNIA MEXICAN GRILL
DESCANSO GARDEN SPA
AL WAZIR CHICKEN
Missing Name...
LITTLE EASY
PACO'S CANTINA INC
POQUITO MAS
THE BOWERY
EL AMIGO
DOG HAUS CARSON
THE STATION
THE VILLAGE
SUBMARINA
VALLEY SUPER-BURGER
Missing Name...
LAS MOLENDERAS
MIDORI SUSHI RESTAURANT
FUJISAN SUSHI @ SMART & FINAL
LASCARI'S DELI
SABROSITAS
RED PALACE II
KEE WAH BAKERY
RIGO'S PLACE
SANTA MONICA FARMS JR
PIEOLOGY
YUI
WOLFDOWN
CHARLIE'S FISH MARKET
THE GREAT AMERICAN GRILL
STARBUCKS
HAIFA RESTAURANT
CASITA TACO AL CARBON
EL GARABATO
LOUISIANA FRIED CHICKEN
VILLAGE MARKET & LIQUOR
TRIP
TOM & GEORGE
RED LANTERN CHINESE CUISINE
Missing Name...
LOVE LETTER PIZZA & CHICKEN
RUEN PAIR THAI CUISINE
CHU'S GARDEN CHINESE RESTAUR
GUISADOS AL PATRON
LA MICHOACANA ICE CREAM & SNACKS
PALACIO LATINO NIGHT CLUB
BBQ NIGHT
Missing Name...
YOGURTLAND SOUTH BAY
PRIMO CAFE
DAVE & BUSTER'S
SHINA NO YORU
BEN & JERRY'S
BIG D'S BURGERS
MEGA BODEGA
CAKE BASH STUDIO
CASA GAMINO
MASALA GRILL
SUBWAY
RODRIGU

SALT & STRAW
FUSION SUSHI
SAKURA JAPANESE STEAK HOUSE
BLUE BOTTLE COFFEE
PARK CAFE & GRILL
MENDOCINO FARMS
STARS DRIVE INC
CHRISTY'S DONUTS
THE GREEN CHILE
RAIGOSA CAFE
MONTEREY CONCESSIONS
BEST DONUTS
PIZZA EXPRESS
MONTEBELLO SENIOR CENTER
NAPOLI KOSHER PIZZA
HOLIDAY INN WEST COVINA
AMAPOLA DELI & MARKET
RASPADO EXPRESS
TUMMY STUFFER
THE GREAT ROOM CAFE
LADY DONUTS
BREAKERS KOREAN  BBQ
ANTOJITOS MEXICANO PAISA
EVERYDAY CAFE
K&A CATON CHINESE REST.
PEPPERDINE CONFERENCE CAFE
DARLING DONUTS
BLACKDOG COFFEE
A CAFE
GUADALAJARA CITY
CANDLE LIGHT BAKERY
Missing Name...
AHI HAWAIIAN BBQ
RESTAURANT MEXICO 86
BELCAMPO MEAT CO
THE COVE
ANA'S BIONICOS
DEFRANKO'S
L&L HAWAIIAN BBQ
SPITFIRE II
MITSURU CAFE
WINGSTOP
GARDEN WOH
G&K BURGERS
OLYMPIC DONUT SHOP
HOUSE OF COOKIES
DOWNTOWN WOMENS CTR KITCHEN
LITTLE CAESARS
MERENGUE BAKERY & CAFE
WING STOP
GUISADOS
HALAL OGLU
SNOWFLAKES SHAVED ICE
VERVE COFFEE
BIA CAFE
CHENGDU HOUSE
Missing Name...
THE SMOKE HOUSE
THE FLAME BROILER
THAI B-B-Q & SEAFOOD REST

NATIONAL BAKING CO
ATTICUS
Missing Name...
WABA GRILL
THE KICKIN CRAB EXPRESS
BUTCHER'S DOG
JUGOS NATURALES
CHATEAU LEMONGRASS
EL SIPOTE
STUFT PIZZA
PARIS BAGUETTE
THE LA HOTEL DOWNTOWN
BUFFET GRILL
CASH BOX KTV
ICUE CAFE- COLLEGE OF THE CANYONS
BOBAMIA
BURGERIM
THE SALVATION ARMY
Missing Name...
AZABU SABO
POPEYES CHICKEN
2ND STORY
AGUA AQUI
KOORI
K'GRILL
HAVANA MANIA
TAQUERIA MEXICO
CARNICERIA LATINA
TASTY GOODY CHINESE FOOD
SHUTTERS ON THE BEACH
CHICKEN NOW
OUTTAKES
VFW POST 1944
Missing Name...
THE BRICK MARKET AND DELI
RESTAURANT MARIA
THE RITZ CARLTON
PEKING WOK
LITTLE CAESARS PIZZA
ALL STAR DONUTS
UN RINCONCITO ARGENTINO
TSURUMARU UDON HONPO
PINKBERRY
SPRING TIME IN NY
YOGURTLAND
WALTER'S COFFEE SHOP
ART'S SUBS
SUBWAY 959
Missing Name...
CANTALOOP
THAI VEGAN
HOOTERS RESTAURANTS
LITTLE TOKYO RESTAURANT
SHIN RAMEN
EASTERN FISH MARKET
VICENTE FOODS-DELI
WOOD FIRE PIZZA
GWEN'S SPECIALTY CAKES/CATER
SUSHI ON BRAND
NEW ORLEANS SNO BALLS
PITCHER HOUSE UPPER DECK
JIMMY'S FAMOUS AMERICAN

MR ANGELOS PIZZA
POTATO CORNER
YUKO KITCHEN
EL POLLO DORADO
DIN TAI FUNG
CANOGA PARK SENIOR CITIZEN
ST. ANTHONY'S PARISH CENTER
ROSA'S KITCHEN
DAIWA SUSHI
IL PICCOLO RITROVO
7180 SUNSET HOLLYWOOD, LLC
JOY
TACOS MICHOACAN
BOMBAY PALACE
SOUND NIGHTCLUB
THE APPETIZER
MICKEY'S DELI
KING JERKEY
JIN KOOK KOREAN RESTAURANT
UTRO'S AT THE WHARF
NOODLE WORLD JR.
NOODLE WORLD
LINK N HOPS
S AND W COUNTRY DINER
SCHOONER BAR & GRILL
SUSHI TIME
MORELOS RESTAURANT
EGGS N THINGS
WAREHOUSE PIZZA
SPUDNUTS DONUTS
PINKBERRY
LA PIZZA DEL SOL
TERIYAKI MADNESS
THE ORIGINAL RINALDI'S DELI AND CAFE
SPROUTS FARMERS MARKET
ROYAL TEA
CAFE CREOLE
Missing Name...
ROUNDING
BEARD PAPA
CHUYS TACOS & BIONICOS
DUNKIN' DONUTS
CHINA BUFFET
SONORA FOODS
THE GREAT WALL
SNOW FOX
FRESH BROTHERS PIZZA
SIAM BAY RESTAURANT
RITTER'S STEAM KETTLE COOKING
Missing Name...
THAI PLATE RESTAURANT
DUNES RESTAURANT
PINEAPPLE HILL SALOON GRILL
THE GOOD CAKE COMPANY
COPYCAT CAFE
BASKIN ROBBINS
SPICY CHARM
THE KETTLE
JERSEY MIKE'S SUBS
MARIS

In [9]:
final_df['Restaurant_Name'] = name
final_df['Rating'] = rating
final_df['Review_Count'] = review_count
final_df['Price'] = price
final_df.to_csv("Yelp_Raw_Data_181216.csv", index=False)



In [10]:
final_df['Review_Count'] = pd.to_numeric(final_df['Review_Count'],errors='coerce').fillna(0).astype(np.int64)
q1 = np.percentile(final_df['Review_Count'],25)

final_df2 = final_df.loc[final_df['Review_Count'] > q1, :]

final_df2.to_csv("Yelp_Cleaned_Data_181216.csv", index=False)



In [17]:
final_df_with_rating = final_df


In [28]:
sample_df["SCORE"] = sample_df['SCORE'].tolist()

In [31]:
sample_df = sample_df.drop(columns = ['Score'])
sample_df

,ACTIVITY DATE,OWNER NAME,FACILITY ID,FACILITY NAME,PROGRAM STATUS,PE DESCRIPTION,FACILITY ADDRESS,FACILITY CITY,FACILITY ZIP,SCORE,GRADE,FULL ADDRESS
0,2018-08-10,"CHENG, FLORA",FA0042891,PAGODA INN,ACTIVE,RESTAURANT (61-150) SEATS HIGH RISK,19348 RINALDI ST,PORTER RANCH,91326,70,C,"19348 RINALDI ST, PORTER RANCH, CA 91326"
1,2018-07-24,GLORIA BURGOS,FA0202740,TACOS EL SINALOENSE,ACTIVE,RESTAURANT (0-30) SEATS HIGH RISK,4813 E ROSECRANS AVE,COMPTON,90221-1851,87,B,"4813 E ROSECRANS AVE, COMPTON, CA 90221-1851"
2,2017-09-19,BOULEVARD LLC,FA0051336,THE DIME,ACTIVE,RESTAURANT (31-60) SEATS LOW RISK,442 N FAIRFAX AVE,LOS ANGELES,90036,93,A,"442 N FAIRFAX AVE, LOS ANGELES, CA 90036"
3,2018-07-17,ELITA KACHATRYAN/ARMINE SARGSY,FA0177733,AROMA BAKERY,ACTIVE,RESTAURANT (0-30) SEATS MODERATE RISK,12502 1/2 VANOWEN ST,NORTH HOLLYWOOD,91605,90,A,"12502 1/2 VANOWEN ST, NORTH HOLLYWOOD, CA 91605"
4,2018-06-13,GREAT AMERICAN CHICKENT CORP,FA0033130,KFC/TACO BELL,ACTIVE,RESTAURANT (31-60) SEATS MODERATE RISK,2941 W IMPERIAL HWY,INGLEWOOD,90303,95,A,"2941 W IMPERIAL HWY, INGLEWOOD, CA 90303"
5,2018-03-19,NORTAME INC,FA0251273,SUSHIMON,ACTIVE,RESTAURANT (31-60) SEATS HIGH RISK,3948 SEPULVEDA BLVD,TORRANCE,90505,91,A,"3948 SEPULVEDA BLVD, TORRANCE, CA 90505"
6,2018-08-14,EDWARD THOMAS HOSPITALITY CO,FA0008073,HOTEL CASA DEL MAR CATCH RESTAURANT,ACTIVE,RESTAURANT (61-150) SEATS HIGH RISK,1910 OCEAN FRONT WALK,SANTA MONICA,90401,96,A,"1910 OCEAN FRONT WALK, SANTA MONICA, CA 90401"
7,2018-07-19,"A & B AZUSA, INC",FA0249603,GRILL CITY - PREP KITCHEN,ACTIVE,RESTAURANT (0-30) SEATS HIGH RISK,2700 COLORADO BLVD,LOS ANGELES,90041,94,A,"2700 COLORADO BLVD, LOS ANGELES, CA 90041"
8,2018-07-09,CHOI JOON JAE,FA0035922,LE CAFE NOIR,ACTIVE,RESTAURANT (0-30) SEATS MODERATE RISK,500 S VIRGIL AVE STE 105,LOS ANGELES,90020,96,A,"500 S VIRGIL AVE STE 105, LOS ANGELES, CA 90020"
9,2018-08-30,SERGIO PAVON,FA0259294,THE FAMILY CAFE,ACTIVE,RESTAURANT (0-30) SEATS HIGH RISK,12121 WILSHIRE BLVD # 110,LOS ANGELES,90025,94,A,"12121 WILSHIRE BLVD # 110, LOS ANGELES, CA 90025"


In [22]:
final_df_with_rating["Score"] = sample_df

In [24]:
bins = [0, 80, 90, 100]

grade = ["C", "B", "A"]

final_df_with_rating["Grade"] = pd.cut(final_df_with_rating["Score"], bins, labels=grade)

final_df_with_rating

,Restaurant_Name,Rating,Review_Count,Price,Score,Grade
0,Pagoda Inn,4,251,$$,70,C
1,Tacos El Sinaloense,4,35,$,87,B
2,The Dime,3.5,319,$$,93,A
3,Aroma Bakery,4,14,$,90,B
4,KFC,1.5,24,$,95,A
5,Sushimon,4,192,$,91,A
6,Catch,3.5,229,$$$,96,A
7,Max City BBQ,4,818,$$,94,A
8,Le Cafe Noir,4.5,20,$,96,A
9,Pho Family Restaurant,4,101,$,94,A


In [33]:
final_df_with_rating['Review_Count'] = pd.to_numeric(final_df_with_rating['Review_Count'],errors='coerce').fillna(0).astype(np.int64)
q1 = np.percentile(final_df_with_rating['Review_Count'],25)

final_df3 = final_df_with_rating.loc[final_df_with_rating['Review_Count'] > q1, :]

final_df3.to_csv("Final_Dataset_181216.csv", index=False)

In [34]:
final_df3

,Restaurant_Name,Rating,Review_Count,Price,Score,Grade
0,Pagoda Inn,4,251,$$,70,C
2,The Dime,3.5,319,$$,93,A
5,Sushimon,4,192,$,91,A
6,Catch,3.5,229,$$$,96,A
7,Max City BBQ,4,818,$$,94,A
9,Pho Family Restaurant,4,101,$,94,A
10,Commonwealth Restaurant,4.5,770,$$$,93,A
11,Emon,4,132,$$,90,B
12,Tacos El Zorro,4,78,$,93,A
13,Agoura Hills Numero Uno,4,112,$$,97,A


In [40]:
pprint(yelp_data)

{'businesses': [{'alias': 'juanitos-place-lancaster',
                 'categories': [{'alias': 'mexican', 'title': 'Mexican'}],
                 'coordinates': {'latitude': 34.7041149,
                                 'longitude': -118.130705},
                 'display_phone': '(661) 948-8811',
                 'distance': 20.271545270368136,
                 'id': 'T5MtwqKhQnnJVuSSxIYiJA',
                 'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/TzSktqzzcbPDH2mQisV90w/o.jpg',
                 'is_closed': False,
                 'location': {'address1': '105 W Ave I',
                              'address2': '',
                              'address3': '',
                              'city': 'Lancaster',
                              'country': 'US',
                              'display_address': ['105 W Ave I',
                                                  'Lancaster, CA 93534'],
                              'state': 'CA',
                              'zip

In [69]:
headers = {"Authorization": "Bearer "+ yelp_key}
target_url = "https://api.yelp.com/v3/businesses/search"

category = []

for index2, row2 in sample_df.iterrows():
    params = {"term": row2["FACILITY NAME"], "location": row2["FULL ADDRESS"]}
    
    yelp_response = requests.get(target_url, params, headers=headers)
    
    if yelp_response.status_code != requests.codes.ok:
        print(yelp_response.text)
    else:
        yelp_data = yelp_response.json()
        
        try:
            category.append(yelp_data['businesses'][0]['categories'][0]['title'])
            print(row2['FACILITY NAME'],index2)
        except (KeyError, IndexError):
            print("Missing title...", index2)
            category.append("na")
    time.sleep(.06)

PAGODA INN 0
TACOS EL SINALOENSE 1
THE DIME 2
AROMA BAKERY 3
KFC/TACO BELL 4
SUSHIMON 5
HOTEL CASA DEL MAR CATCH RESTAURANT 6
GRILL CITY - PREP KITCHEN 7
LE CAFE NOIR 8
THE FAMILY CAFE 9
COMMONWEALTH 10
EMON JAPANESE RESTAURANT 11
EL ZORRO NIGHTCLUB 12
NUMERO UNO AGOURA HILLS 13
C & O TRATTORIA 14
THE BLVD 15
CAESAR PALACE 16
CAL MARE 17
LA FAVORITA BAKERY 18
HUDSON HOUSE 19
CAFE TRES LECHES 20
TACO BELL 21
BOILING POINT RESTAURANT HACIENDA HEIGHTS 22
LA INDIANA RESTAURANT 23
SUBWAY 24
CHA CHA CHICKEN 25
FREEZZONE RASPADOS 26
RAINBOW DONUTS 27
Missing title... 28
RED IRON MONGOLIAN BBQ 29
THAI TIFFANY 30
SODEXHO MANAGEMENT INC 31
RINALDI'S OF REDONDO ITALIAN 32
Missing title... 33
SAN YA BBQ RESTAURANT 34
INDIA'S GRILL 35
RINJANI 36
GREEN COTTAGE RESTAURANT 37
CASSIA RESTAURANT 38
Missing title... 39
KEIKO GRILL 40
SCHOONERVILLE 41
R & B ROLL & KATSU 42
M & A AROMA 43
SUSHI RESTAURANT TAIHEI 44
SLOOPY'S BEACH CAFE 45
SUBWAY 46
THE RABBIT HOLE 47
Missing title... 48
CHINA WOK 49
MISS DO

HOT POT LAND 396
PHO SAIGON 397
SHAMROCK LOUNGE 398
THAI ME UPP 399
OLD PLACE 400
WENDY'S DONUTS 401
SCHWARTZ BAKERY 402
TAM'S BURGER 403
PEPES G. TACOS 404
CYPRESS BEST BURGERS 405
VERMONT MUSIC STUDIO 406
MINIBAR HOLLYWOOD 407
MELODY 408
THE CORK 409
LUCY'S PLACE 410
ISLANDS FINE BURGERS & DRINKS 411
PAPA JOHN'S PIZZA 412
PEET'S COFFEE & TEA 413
Missing title... 414
326 BEER AND WINE 415
PIENCONE 416
RESIDENT 417
HOLLYWOOD PIES 418
SEAFOOD PALACE 419
YAMADAYA 420
EL APPETIZ 421
STARBUCKS 422
SWIRLZ SOFT SERVE PARLOR 423
EL TAPATIO GRILL AND BAKERY 424
YARD HOUSE 425
LOCK & KEY SOCIAL DRINKERY 426
THE STONE 427
THE DEN 428
NAPOLE 429
SHAHERZARD RESTAURANT 430
WHOLE FOODS MARKET 431
DELANCEY 432
Missing title... 433
B'S LAND 434
BULACAN  BAKERY AND CATERING 435
CARNITAS LOYA FAST FOOD 436
HOWLING JUICE 437
GOLDEN BULL 438
BEST DONUTS AND ICE CREAM 439
Missing title... 440
GRUNIONS SPORTS BAR 441
PANDA EXPRESS 442
SHORT STOP SEAFOOD BAR 443
COCO FRESH TEA & JUICE 444
GRILL CITY / BUBBLE

YELLOW VASE 778
SIR PIZZA 'N' CHICKEN 779
LA METRO 780
YOSHINOYA BEEF BOWL 781
TIJUANAS TACOS 782
FONDA DON CHON 783
SWEET LADY JANE 784
THE FLAME BROILER 785
STARBUCKS 786
BARREL 787
ROUND TABLE PIZZA 788
LOS TRES COCHINITOS 789
GUNGJUNG SULRUNGTANG 790
KREATION JUICERY 791
EL PEDREGAL 792
SECOND CITY BISTRO 793
B20 BAR & GRILL RESTAURANT 794
CALIFORNIA ROLL FACTORY 795
CAPRI DELI & KITCHEN INC 796
BERNADETT'S 797
FOOTHILL CINEMA 10 798
BASTARDS BAB-B-QUE 799
COACH'S BAR & GRILL 800
BASKIN ROBBINS 801
CAROLYN ROSAS PARK 802
DELLA TERRA 803
LITTLE CAESARS 804
PINCHES TACOS 805
MOLLY MALONES 806
HOT POINT HOT POT III 807
EL CAPIRO CLUB 808
MELT GELATO & CREPES CAFE 809
JERSEY MIKE'S SUBS 810
THE COFFEE BEAN & TEA LEAF 811
CHOURA VENUE SERVICES 812
PALISADES PIZZA 813
CEMITAS POBLANAS MI MAGDALENA 814
NOBU LOS ANGELES 815
PHAT BIRDS 816
FACTORS FAMOUS DELI 817
TACOS EL ZORRO 818
BOILING POINT ROWLAND HEIGHTS 819
EL CHALATECO 820
JERSEY MIKE'S SUBS 821
BABALU CAFE 822
MONTY 823
IL BUCO RE

CHEMA'S TACOS 1157
RESTAURANTE TIJUANA 1158
HASH 1159
Missing title... 1160
NATUREWELL 1161
LUCKY DONUTS 1162
SASSAFRAS 1163
LAS PALMAS RESTAURANT 1164
JERSEY MIKE'S SUBS 1165
EAGLES NEST SALOON 1166
BLACK ELEPHANT COFFEE 1167
PAPA JOHNS PIZZA 1168
BUCHONES BAR 1169
GALBI BROTHERS INC / BROTHERS B.B.Q. 1170
BRUDDAH'S BAR & GRILL 1171
SUBWAY SANDWICHES & SALADS 1172
KAPOOR'S AKBAR 1173
POPEYES CHICKEN 1174
GRILL EM ALL 1175
JERSEY MIKE'S SUBS 1176
LILY'S CAKE SI & BAKERY 1177
Missing title... 1178
LUPE'S PLACE 1179
TOGO'S WESTCHESTER 1180
BIG AL'S PIZZA 1181
Missing title... 1182
TACO LIMON 1183
THAI DISHES 1184
ROLLING HILLS COUNTRY CLUB 1185
Missing title... 1186
CAPTAIN TONYS PIZZA & PASTA  EMPORIUM 1187
Missing title... 1188
HOLY GUACAMOLE 1189
NARA MEDITERRANEAN BISTRO & LOUNGE 1190
LUMKANAAD 1191
NIKO NIKO SUSHI 1192
BOBA LOCA 1193
CHINA EXPRESS 1194
THE COFFEE BEAN & TEA LEAF 1195
KITCHEN STORY 1196
BAMBOU LE PHO 1197
VIC'S TACOS 1198
FIESTA MEXICANA EL PAISANO 1199
TAQUERIA EL P

THE FLAME BROILER 1528
ORGINAL THAI 1529
ELEGANT EVENT DECOR & CATERI 1530
CACTUS MEXICAN GRILL 1531
VALERIOS TROPICAL BAKESHOP 1532
NATE-N-AL 1533
Missing title... 1534
VERVE COFFEE ROASTERS 1535
MONTRESOR 1536
Missing title... 1537
RASPADOS Y BIONICOS LAS DELI 1538
ALANA'S COFFEE ROASTING 1539
NOODLE STREET 108 1540
CATHERINE'S CAKE DECORATING & CANDY SUPPLIES 1541
GAUCHO GRILL DOWNEY 1542
Missing title... 1543
ROLADIN BAKERY & CAFE 1544
WABA GRILL 1545
PHILLIPS BAR-B-QUE II 1546
BANCHAN A LA CARTE 1547
ONISHICHO GRILL EXPRESS 1548
VILLA CATRINA MEXICAN RESTAURANT 1549
NANJING SALTED DUCK 1550
TOMMY'S WORLD FAMOUS HAMBURG 1551
CONI SEAFOOD 1552
MAGGIE'S BLUE ROSE 1553
PUMARIN THAI 1554
CANYON CITY BARBEQUE 1555
THE MEAT CELLAR 1556
SWEET ROSE CREAMERY 1557
MEDITERRANEAN TRIANGLE 1558
DINER ON MAIN 1559
KOZAN 1560
EL PARAISO BAKERY 1561
LAWNDALE BREWHOUSE 1562
MARIX TEX MEX 1563
BANANA BAY RESTAURANT 1564
JUICY LADIES 1565
RODINI PARK 1566
BAAN THAI 1567
JONATHAN CLUB 1568
WOODLAND HI

...

STARBUCKS KIOSK-AMERICAN HONDA 3010
SWEETS N TREATS 3011
AL BASCH CHICKEN 3012
SEASIDE AT THE PIER 3013
PLATE & PETAL 3014
PINK ORCHID BAKERY AND CAFE 3015
CUATRO CAMINOS RESTAURANT 3016
CBS BROADCAST CENTER COMMIS 3017
LOS FELIZ CAFE 3018
THE RUSTEN HOUSE 3019
MCDONALD'S 3020
FOSTER'S FREEZE 3021
ARO LATIN 3022
CALIFORNIA DONUTS 3023
THE HABIT BURGER GRILL 3024
CHARLEY'S PHILLY STEAKS 3025
BARNES & NOBLE BOOKSELLERS 3026
Missing title... 3027
BAR NINETEEN 12 3028
SUPER KING MARKET-BAKERY 3029
SWEET BUTTER 3030
KFC/LONG JOHN SILVER 3031
SANTA MONICA YACHT CLUB 3032
FISHING BOAT CHINESE CUISINE 3033
DICKEYS BARBECUE PIT 3034
WIENERSCHNITZEL 679 3035
DOTS CAFE 3036
SHAWN CAFE 3037
CAFE FUSION 3038
MAGIC TOUCH 3039
VINO VOLO 3040
LA BUFADORA BAR 3041
TERI, BURGER 3042
STUFFED SANDWICH 3043
Missing title... 3044
SAN MARINO GRILL 3045
EL HATUCHAY RESTAURANT 3046
RIVERA'S 3047
WABA  GRILL 3048
JINKY'S CAFE 3049
A & W SEAFOOD RESTAURANT 3050
ALEJO'S PRESTO TRATTORIA 3051
TIERRA MIA COFFEE COM

PIEOLOGY 3383
YUI 3384
WOLFDOWN 3385
CHARLIE'S FISH MARKET 3386
THE GREAT AMERICAN GRILL 3387
STARBUCKS 3388
HAIFA RESTAURANT 3389
CASITA TACO AL CARBON 3390
EL GARABATO 3391
LOUISIANA FRIED CHICKEN 3392
VILLAGE MARKET & LIQUOR 3393
TRIP 3394
TOM & GEORGE 3395
RED LANTERN CHINESE CUISINE 3396
Missing title... 3397
LOVE LETTER PIZZA & CHICKEN 3398
RUEN PAIR THAI CUISINE 3399
CHU'S GARDEN CHINESE RESTAUR 3400
GUISADOS AL PATRON 3401
LA MICHOACANA ICE CREAM & SNACKS 3402
PALACIO LATINO NIGHT CLUB 3403
BBQ NIGHT 3404
Missing title... 3405
YOGURTLAND SOUTH BAY 3406
PRIMO CAFE 3407
DAVE & BUSTER'S 3408
SHINA NO YORU 3409
BEN & JERRY'S 3410
BIG D'S BURGERS 3411
MEGA BODEGA 3412
CAKE BASH STUDIO 3413
CASA GAMINO 3414
MASALA GRILL 3415
SUBWAY 3416
RODRIGUEZ BOTANAS 3417
Missing title... 3418
SKYLIGHT GARDENS 3419
LE BISTRO CAFE, INC. 3420
HONEY BAKERY 3421
KLAT 3422
THE PARLOR HOLLYWOOD 3423
BBQ INDIAN CUISINE 3424
SIDECAR DOUGHNUTS AND COFFEE 3425
HEALTHY MEAT BOWL 3426
AROMA BAKERY CAFE 3427


CAESAR'S MONGOLIAN BAR-B-Q 3747
SHAMSHIRI GLENDALE 3748
DRAGO CENTRO-BASEMENT STORAG 3749
RIO'S PIZZA 3750
RENAUD'S PATISSERIE AND BISTRO 3751
TROIS MECS 3752
ALFRED COFFEE STUDIO CITY 3753
MONKEY BAR 3754
Missing title... 3755
BASKIN ROBBINS 3756
KINGS PANADERIA ,INC 3757
TERESA'S MEXICAN GRILL 3758
KASTURI RESTAURANT AND MARKET 3759
SAKE HOUSE 3760
EMERALD THAI CUISINE 3761
GOLDS GYM 3762
UNCLE HENRY'S DELI 3763
TASTY THAI CAFE 3764
BIG WANG'S 3765
PUPUSERIA JUTIAPA 3766
EL LLANO 3767
LAKE LINDERO COUNTRY CLUB 3768
ME-N-ED'S 3769
ING DOI THAI KITCHEN 3770
PINOY FOOD REPUBLIC 3771
DOWNEY ARCO 3772
Missing title... 3773
TENDER GREENS 3774
HOME MADE DONUTS 3775
THE CHEESECAKE FACTORY 3776
ORIENTAL WOK 3777
JOSS BITES 3778
CASA CORDOBA 3779
ZAFRAN POT 3780
THE MUNCHIES SPOT 3781
Missing title... 3782
THE TRUMAN HOUSE 3783
Missing title... 3784
MENCHIE'S  YOGURT SHERMAN OAK 3785
HT GRILL 3786
SOLAR DE CAHUENGA 3787
AWOOLIM KIMBAP FACTORY 3788
MAKI SUSHI 3789
Missing title... 3790
SHAKE SH

CHAYA VENICE 4118
SUBWAY 4119
54TH FLOOR KITCHEN 4120
BARBETTE 4121
Missing title... 4122
SAIGON EDEN, VIETNAMESE REST 4123
LUCY'S EL ADOBE RESTAURANT 4124
LUCKY CHINA 4125
PIZZA MAN 4126
KRUST 4127
KASHIWA RESTAURANT 4128
CALIFORNIA CLUB-MAIN KITCHEN 4129
HEIRLOOM BAKERY & CAFE 4130
AMOR INDIO 4131
RICCO'S PIZZA 4132
CAKES UNLIMITED 4133
THE BURGERSHOP 4134
HAPPY DONUTS 4135
MENCHIE'S FROZEN YOGURT 808 4136
CHIPOTLE 4137
VILLA MORELIANA 4138
CABO CANTINA 4139
LACHA SOMTUM 4140
CUPID'S HOT DOGS 4141
BON APPETIT MGT COMPANY 4142
CAFE VIDA 4143
KUBLAI KHAN 4144
Missing title... 4145
JAMBA JUICE 4146
STARBUCKS -  DEPARTURE 4147
BIONICOS JUGOS ZAPOPAN 4148
PERCOLATE 4149
WON KOK RESTAURANT 4150
Missing title... 4151
HOCK AND HOOF 4152
KATIE'S KITCHEN 4153
PINK PEPPER 4154
CALAMIGOS 4155
COLDSTONE CREAMERY 4156
BROADWAY GRILL 4157
THE HABIT BURGER GRILL 4158
TOGO'S & BASKIN ROBBINS 4159
HUNGRY CROWD 4160
VILLAGE FRENCH BAKERY 4161
TUTTI FRUTTI NORWALK 4162
TACOS EL GORDO 4163
Missing title.

ConnectionError: HTTPSConnectionPool(host='api.yelp.com', port=443): Max retries exceeded with url: /v3/businesses/search?term=BUTCHER%27S+DOG&location=11301+W+OLYMPIC+BLVD%2C+LOS+ANGELES%2C+CA+90064 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x117edf780>: Failed to establish a new connection: [Errno 60] Operation timed out',))

In [53]:
final_df3[final_df3['Restaurant_Name'] == 'Bottega Louie']

,Restaurant_Name,Rating,Review_Count,Price,Score,Grade
195,Bottega Louie,4,15786,$$,97,A
935,Bottega Louie,4,15786,$$,91,A
1177,Bottega Louie,4,15786,$$,97,A
1450,Bottega Louie,4,15786,$$,96,A
1679,Bottega Louie,4,15786,$$,91,A
1729,Bottega Louie,4,15786,$$,92,A
1980,Bottega Louie,4,15786,$$,90,B
2310,Bottega Louie,4,15786,$$,90,B
4869,Bottega Louie,4,15786,$$,97,A
4979,Bottega Louie,4,15784,$$,95,A


In [62]:
final_df3['Restaurant_Name'].nunique()

3196

In [63]:
final_df3.count()

Restaurant_Name    3735
Rating             3735
Review_Count       3735
Price              3735
Score              3735
Grade              3735
dtype: int64

In [64]:
final_df4 = final_df3
duplicated_df = pd.DataFrame()
final_df4['duplicated'] = final_df3.duplicated('Restaurant_Name')

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [67]:
final_df4.duplicated('Restaurant_Name')

0    False
2    False
5    False
6    False
7    False
dtype: bool

In [68]:
final_df4

,Restaurant_Name,Rating,Review_Count,Price,Score,Grade,duplicated
0,Pagoda Inn,4,251,$$,70,C,False
2,The Dime,3.5,319,$$,93,A,False
5,Sushimon,4,192,$,91,A,False
6,Catch,3.5,229,$$$,96,A,False
7,Max City BBQ,4,818,$$,94,A,False
9,Pho Family Restaurant,4,101,$,94,A,False
10,Commonwealth Restaurant,4.5,770,$$$,93,A,False
11,Emon,4,132,$$,90,B,False
12,Tacos El Zorro,4,78,$,93,A,False
13,Agoura Hills Numero Uno,4,112,$$,97,A,False


In [ ]:
final_df4 = final_df4.drop